# Naive Bayes Assignment Tobigs15th 이성범

## 우리의 목표 : 나이브베이즈 문제 해결하기
1.1) 입력문서가 {fast, furious, fun} 만을 주요 단어로 가질때, 이 문서는 얼마의 확률로 어떤 문서로 분류되는가?

##### 노트북 파일을 따라오면서, 빈칸과 질문에 대한 물음을 모두 채우시면 과제 완료 입니다 

### 주어진 dataset 을 확인해 보기위해 엑셀 파일을 읽습니다.

In [2]:
import numpy as np
import pandas as pd

In [3]:
documents = pd.read_excel('test_file.xlsx')

In [4]:
documents

,label,message
0,comedy,"Fun couple, love love"
1,action,"fast Furious, shoot!!"
2,comedy,"couple^^, fly, fast, fun, fun"
3,action,"furious, shoot shoot fun"
4,action,"fly~~ fast shoot, Love"


####  Q1. **문제점**(전처리가 필요한 부분)을 2가지 이상 적어주세요.
- 1. ^^, ~~ 등의 특수문자가 들어있다.
- 2. 문자의 구분이 " "인 경우도 있고 ","도 있다.
- 3. 파이썬의 경우 대문자와 소문자를 다르게 인식하는데 위 경우에는 대문자와 소문자가 혼합되어 있다.

### step 1. 단어를 모두 소문자로 바꾸어 줍니다. 

#### Q2. 코드속 빈칸을 채워주세요

In [5]:
lower_documents = []
# lower() 함수를 통해서 소문자로 변환해주었다.
lower_documents = [d.lower() for d in documents['message']]
print(lower_documents)

['fun couple, love love', 'fast furious, shoot!!', 'couple^^, fly, fast, fun, fun', 'furious, shoot shoot fun', 'fly~~ fast shoot, love']


### step2. 특수문자를 없애줍니다.
#### string.punctuation 에는 특수문자들이 저장되어있는것을 확인할수 있습니다.

In [16]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

### Q3. string.punctuation 을 이용해서 특수문자를 제거해주세요.

In [17]:
import re

del_punctuation_documents = []

for i in lower_documents:
    # 정규표현식을 사용해주었다.
    # [ ] 안에 들어있는 문자와 i와 매칭되는 것이 있다면 ""로 변환해준다는 것을 의미한다.
    # [ ] 안에는 모든 특수문자가 들어있다.
    del_punctuation_documents.append(re.sub(f'[{string.punctuation}]', '', i))
    
del_punctuation_documents

['fun couple love love',
 'fast furious shoot',
 'couple fly fast fun fun',
 'furious shoot shoot fun',
 'fly fast shoot love']

### step 3. 단어를 하나씩 띄어쓰기 단위로 쪼개어 줍니다.
#### Q4. 빈칸을 채워주세요.

In [18]:
# 문자를 " " 기준으로 분리해줬다.
preprocessed_documents = [[w for w in d.split()] for d in del_punctuation_documents]
preprocessed_documents

[['fun', 'couple', 'love', 'love'],
 ['fast', 'furious', 'shoot'],
 ['couple', 'fly', 'fast', 'fun', 'fun'],
 ['furious', 'shoot', 'shoot', 'fun'],
 ['fly', 'fast', 'shoot', 'love']]

### step 4. 각각의 단어가 문서에서 몇번 나왔는지 세봅시다.

In [19]:
frequency_list = []
from collections import Counter

# 하나의 리스트에서 등장하는 단어의 수를 세주어 반환해주었다.
frequency_list = [Counter(d) for d in preprocessed_documents]
frequency_list

[Counter({'fun': 1, 'couple': 1, 'love': 2}),
 Counter({'fast': 1, 'furious': 1, 'shoot': 1}),
 Counter({'couple': 1, 'fly': 1, 'fast': 1, 'fun': 2}),
 Counter({'furious': 1, 'shoot': 2, 'fun': 1}),
 Counter({'fly': 1, 'fast': 1, 'shoot': 1, 'love': 1})]

### step 5. 이제 우리가 하고싶은것은, 문자를 숫자로 변환하는 작업입니다!(그림 참고)
- 이를 위해 CountVectorizer를 사용합니다.
- CountVectorizer는 문서 집합에서 단어 토큰을 생성하고, 각 단어 수를 세어 BOW (Bag of Words) 인코딩한 벡터를 만들어줍니다.

- 문서에 해당단어가 몇번 포함되었는지를 나타낼 때 사용하는 방법입니다.

- 참고 : https://datascienceschool.net/view-notebook/3e7aadbf88ed4f0d87a76f9ddc925d69/

![CountVectorized](https://user-images.githubusercontent.com/68625698/106378540-15d8ed80-63e9-11eb-8604-5c960c274867.PNG)

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()
# documents['message']에 들어있는 모든 문장을 학습한다.
count_vector.fit(documents['message'])
# documents['message']에 들어있는 모든 문장을
# 특수문자를 제외하고 뛰어쓰기를 기준으로 단어를 분리해준다.

CountVectorizer()

In [21]:
doc_array = count_vector.transform(documents['message']).toarray()
doc_array
# documents['message']에 들어있는 모든 문장으로 학습한 결과의 단어들을
# 하나의 문장에 존재하는지 count히여 BOW 인코딩을 해준 결과이다.

array([[1, 0, 0, 1, 0, 2, 0],
       [0, 1, 0, 0, 1, 0, 1],
       [1, 1, 1, 2, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 2],
       [0, 1, 1, 0, 0, 1, 1]], dtype=int64)

In [22]:
count_vector.get_feature_names()
# 분리된 단어를 볼 수 있다.

['couple', 'fast', 'fly', 'fun', 'furious', 'love', 'shoot']

In [23]:
count_vector.vocabulary_
# doc_array 배열에 순서에 따라 매치되는 단어를 나타낸다.
# ex) couple의 경우 0열에 해당한다고 볼 수 있다.

{'fun': 3,
 'couple': 0,
 'love': 5,
 'fast': 1,
 'furious': 4,
 'shoot': 6,
 'fly': 2}

In [24]:
frequency_matrix = pd.DataFrame(doc_array, columns = count_vector.get_feature_names())
frequency_matrix
# BOW 인코딩 결과를 DataFrame으로 만들어줬다.

,couple,fast,fly,fun,furious,love,shoot
0,1,0,0,1,0,2,0
1,0,1,0,0,1,0,1
2,1,1,1,2,0,0,0
3,0,0,0,1,1,0,2
4,0,1,1,0,0,1,1


In [25]:
frequency_matrix['count'] = frequency_matrix.sum(axis=1)
# 각 행을 모두 더하여
# 하나의 문장에 존재하는 모든 단어의 수를 구해줬다.
# ex) 0행의 경우 couple 1개, fun 1개, love 2개로 총합이 4가 나온다.
# 위와 같은 방식으로 각 행마다의 총합을 구해준다.

### step 6. 범주형 변수를 dummy변수로 변환해주는 작업(One-Hot Encoding!)을 해주어야합니다.
#### Q5. label 을 comedy =1, action =0 으로 변환해주세요

In [26]:
documents["label"] = documents["label"].map(lambda x : 1 if x == 'comedy' else 0)
# comedy면 1, action면 0으로 원핫인코딩을 해주었다.

In [27]:
doc = pd.concat([documents['label'],frequency_matrix],axis=1)
# documents['label'],frequency_matrix를 하나로 묶어주었디.

In [28]:
doc

,label,couple,fast,fly,fun,furious,love,shoot,count
0,1,1,0,0,1,0,2,0,4
1,0,0,1,0,0,1,0,1,3
2,1,1,1,1,2,0,0,0,5
3,0,0,0,0,1,1,0,2,4
4,0,0,1,1,0,0,1,1,4


###  step 7. 나이브 베이즈 계산을 해봅시다!

#### Q6. 입력문서가 {fast, furious, fun} 을 주요 단어로 가질때, 이 문서는 얼마의 확률로 어떤 문서로 분류가 될까요? ( 계산과정을 채워주세요) 

In [29]:
doc =  doc.values          # dataframe을 np-array로 변환해줍니다.
                           # return값은 np-matrix가 아니라 np-array입니다.
doc

array([[1, 1, 0, 0, 1, 0, 2, 0, 4],
       [0, 0, 1, 0, 0, 1, 0, 1, 3],
       [1, 1, 1, 1, 2, 0, 0, 0, 5],
       [0, 0, 0, 0, 1, 1, 0, 2, 4],
       [0, 0, 1, 1, 0, 0, 1, 1, 4]], dtype=int64)

In [30]:
type(doc)

numpy.ndarray

####  P(Y=comedy), P(Y=action) 계산하기

$ P(Y=comedy) = $ comedy로 분류된 문장의 수 / 전체 문장의 수

$ P(Y=action) = $ action로 분류된 문장의 수 / 전체 문장의 수

In [31]:
# P(Y=comedy)
p_comedy = sum(doc[:,0]==1) / len(doc)

# P(Y=action)
p_action = sum(doc[:,0]==0) / len(doc)

In [32]:
print('p_comedy : ', p_comedy)
print('p_action : ', p_action)

p_comedy :  0.4
p_action :  0.6


#### P(fast=1|comedy=1), P(furious=1|comedy=1), P(fun=1|comedy=1) 계산하기
* 참고 : 문서에 두번 등장한 단어 주의

파이썬은 0열 부터 시작한다는 것을 참고

comedy는 0열에 값이 1이라는 것음 참고

fast은 2열에 위치

fun은 4열에 위치

furious은 5열에 위치

각 행마다 단어의 빈도수의 총합은 count의 열에 존재하며 8열에 위치

$ P(fast=1|comedy=1) = $ comedy로 분류된 문장 중 fast 단어의 수 / comedy로 분류된 문장에 총 단어 수

$ P(furious=1|comedy=1) = $ comedy로 분류된 문장 중 furious 단어의 수 / comedy로 분류된 문장에 총 단어 수

$ P(fun=1|comedy=1) = $ comedy로 분류된 문장 중 fun 단어의 수 / comedy로 분류된 문장에 총 단어 수

In [41]:
# P(fast=1|comedy=1)
p_comedy_fast = sum(doc[(doc[:, 0] == 1) & (doc[:, 2] != 0)][:, 2]) / sum(doc[doc[:, 0] == 1][:, -1])

# P(furious=1|comedy=1)
p_comedy_furious = sum(doc[(doc[:, 0] == 1) & (doc[:, 5] != 0)][:, 5]) / sum(doc[doc[:, 0] == 1][:, -1]) 

# P(fun=1|comedy=1)
p_comedy_fun = sum(doc[(doc[:, 0] == 1) & (doc[:, 4] != 0)][:, 4]) / sum(doc[doc[:, 0] == 1][:, -1])

In [42]:
print('p_comedy_fast : ' , p_comedy_fast)
print('p_comedy_furious : ' , p_comedy_furious)
print('p_comedy_fun : ' , p_comedy_fun)

p_comedy_fast :  0.1111111111111111
p_comedy_furious :  0.0
p_comedy_fun :  0.3333333333333333


#### P(fast=1|action=1), P(furious=1|action=1), P(fun=1|action=1) 계산하기
* 참고 : 문서에 두번 등장한 단어 주의

파이썬은 0열 부터 시작한다는 것을 참고

action는 0열에 값이 0이라는 것을 참고

fast은 2열에 위치

fun은 4열에 위치

furious은 5열에 위치

각 행마다 단어의 빈도수의 총합은 count의 열에 존재하며 8열에 위치

$ P(fast=1|action=1) = $ action로 분류된 문장 중 fast 단어의 수 / action로 분류된 문장에 총 단어 수

$ P(furious=1|action=1) = $ action로 분류된 문장 중 furious 단어의 수 / action로 분류된 문장에 총 단어 수

$ P(fun=1|action=1) = $ action로 분류된 문장 중 fun 단어의 수 / action로 분류된 문장에 총 단어 수

In [43]:
# P(fast=1|action=1)
p_action_fast = sum(doc[(doc[:, 0] == 0) & (doc[:, 2] != 0)][:, 2]) / sum(doc[doc[:, 0] == 0][:, -1])

# P(furious=1|action=1)
p_action_furious = sum(doc[(doc[:, 0] == 0) & (doc[:, 5] != 0)][:, 5]) / sum(doc[doc[:, 0] == 0][:, -1])

# P(fun=1|action=1)
p_action_fun = sum(doc[(doc[:, 0] == 0) & (doc[:, 4] != 0)][:, 4]) / sum(doc[doc[:, 0] == 0][:, -1])

In [44]:
print('p_action_fast : ' , p_action_fast)
print('p_action_furious : ' , p_action_furious)
print('p_action_fun : ' , p_action_fun)  

p_action_fast :  0.18181818181818182
p_action_furious :  0.18181818181818182
p_action_fun :  0.09090909090909091


#### P(Y = comedy| X = fast, furious, fun) , P(Y = action=1| X = fast, furious, fun) 값 계산하기

$ P(Y = comedy| X = fast, furious, fun) = \frac{P(fast=1|comedy=1) P(furious=1|comedy=1) P(fun=1|comedy=1)  P(comedy=1)}{P(X = fast, furious, fun)} $

$ P(Y = action=1| X = fast, furious, fun) = \frac{P(fast=1|action=1) P(furious=1|action=1) P(fun=1|action=1)  P(action=1)}{P(X = fast, furious, fun)} $ 

로 구할 수 있다. 그런데 우리는 proba_comedy와 proba_action의 값을 비교하는 것인데 두 값을 구하는 식의 분모가 동일하기 때문에 분자의 값만 비교해주면 된다.

분자의 값만을 구해도 되는 이유는 우리는 장르만 다를 뿐이지 구하고자 하는 단어는 같기 때문이다. 따라서 분자만을 구하여 두 개의 값을 비교하여 fast, furious, fun 단어가 어느 장르에서 더 많이 나오는지를 알 수 있다.

In [45]:
#P(Y = comedy| X = fast, furious, fun)
proba_comedy = p_comedy * p_comedy_fast * p_comedy_furious * p_comedy_fun

#P(Y = action=1| X = fast, furious, fun)
proba_action = p_action * p_action_fast * p_action_furious * p_action_fun

In [46]:
print('proba_comedy', proba_comedy)
print('proba_action', proba_action)

proba_comedy 0.0
proba_action 0.001803155522163787


### step 8. 다음 값을 비교해봅시다.
1. P(Y = comedy| X = fast, furious, fun)
2. P(Y = action=1| X = fast, furious, fun)


Q7. 어떤 문제점을 발견할수 있나요? 문제점을 해결하기 위한 방법으로는 어떤게 있을까요?
- A. proba_comedy를 보면 p_comedy_furious의 값인 0으로 인해 확률이 0으로 계산되었다는 것을 확인할 수 있다. 이를 해결하기 위해서는 likelihood가 0이 되는 것을 방지하도록 최소한의 확률을 정해주는 라플라스 스무딩 방식이 있다. 라플라스 스무딩 방식은 분자에 + 1 값을 더해줌으로써 최소한의 확률을 정해주고 분모에는 입력변수들의 개수 즉 관측되는 단어의 개수만큼을 더해주는 방식이다.

In [57]:
# P(fast=1|comedy=1)
p_comedy_fast_L = (sum(doc[(doc[:, 0] == 1) & (doc[:, 2] != 0)][:, 2]) + 1) / (sum(doc[doc[:, 0] == 1][:, -1]) + (doc.shape[1] - 2))

# P(furious=1|comedy=1)
p_comedy_furious_L = (sum(doc[(doc[:, 0] == 1) & (doc[:, 5] != 0)][:, 5]) + 1) / (sum(doc[doc[:, 0] == 1][:, -1]) + (doc.shape[1] - 2))

# P(fun=1|comedy=1)
p_comedy_fun_L = (sum(doc[(doc[:, 0] == 1) & (doc[:, 4] != 0)][:, 4]) + 1) / (sum(doc[doc[:, 0] == 1][:, -1]) + (doc.shape[1] - 2))

In [58]:
print('p_comedy_fast_L : ' , p_comedy_fast_L)
print('p_comedy_furious_L : ' , p_comedy_furious_L)
print('p_comedy_fun_L : ' , p_comedy_fun_L)

p_comedy_fast_L :  0.125
p_comedy_furious_L :  0.0625
p_comedy_fun_L :  0.25


In [59]:
# P(fast=1|action=1)
p_action_fast_L = (sum(doc[(doc[:, 0] == 0) & (doc[:, 2] != 0)][:, 2]) + 1) / (sum(doc[doc[:, 0] == 0][:, -1]) + (doc.shape[1] - 2))

# P(furious=1|action=1)
p_action_furious_L = (sum(doc[(doc[:, 0] == 0) & (doc[:, 5] != 0)][:, 5]) + 1) / (sum(doc[doc[:, 0] == 0][:, -1]) + (doc.shape[1] - 2))

# P(fun=1|action=1)
p_action_fun_L = (sum(doc[(doc[:, 0] == 0) & (doc[:, 4] != 0)][:, 4]) + 1) / (sum(doc[doc[:, 0] == 0][:, -1]) + (doc.shape[1] - 2))

In [60]:
print('p_action_fast_L : ' , p_action_fast_L)
print('p_action_furious_L : ' , p_action_furious_L)
print('p_action_fun_L : ' , p_action_fun_L) 

p_action_fast_L :  0.16666666666666666
p_action_furious_L :  0.16666666666666666
p_action_fun_L :  0.1111111111111111


In [61]:
#P(Y = comedy| X = fast, furious, fun)
proba_comedy_L = p_comedy * p_comedy_fast_L * p_comedy_furious_L * p_comedy_fun_L

#P(Y = action=1| X = fast, furious, fun)
proba_action_L = p_action * p_action_fast_L * p_action_furious_L * p_action_fun_L

In [62]:
print('proba_comedy_L : ', proba_comedy_L)
print('proba_action_L : ', proba_action_L)

proba_comedy_L :  0.00078125
proba_action_L :  0.0018518518518518513


두개의 값을 비교하면 action 장르일 때 fast, furious, fun 단어가 더 자주 나온다고 할 수 있다.